In [2]:
import numpy as np
from scipy.stats import chi2, poisson, kstest, norm

### Пункт а)

In [65]:
#  Пункт а)
data = np.array([5, 8, 6, 12, 14, 18, 11, 6, 13, 7])
n = data.sum()

# a) Пирсон
M = np.array([5, 8, 6, 12, 14, 18, 11, 6, 13, 7])
NP = np.array([10] * 10)
print(f'Pirson: delta = {np.sum((NP - M)**2/NP)}')
print(f'p-value = {0.0589}')

# a) Колмогоров
sample = np.repeat(np.arange(10), data)
ks_stat, p_value_ks = kstest(sample, 'uniform', args=(0, 9))

print(f"Kolmogorov: delta = {n**0.5 * ks_stat:.4f}")
print(f'p-value = {0.0328}')

Pirson: delta = 16.4
p-value = 0.0589
Kolmogorov: delta = 1.4333
p-value = 0.0328


### Пункт б)

In [66]:
import numpy as np
from scipy.stats import norm
from functools import reduce
import operator
from scipy.optimize import minimize
#  Ищем максимум для ОМПГ
def normal_distribution_integral(pair, mu, sigma):
    a, b = pair[0], pair[1]
    if sigma <= 0:
        raise ValueError("sigma должно быть > 0")
    return norm.cdf(b, loc=mu, scale=sigma) - norm.cdf(a, loc=mu, scale=sigma)

def product_of_integrals(degres, intervals, mu, sigma):
    # Вычисляем вероятности для каждого интервала и возводим в степень
    probabilities = [normal_distribution_integral(intervals[i], mu, sigma) ** degres[i] for i in range(len(intervals))]
    # Перемножаем все вероятности
    return reduce(operator.mul, probabilities, 1.0)

def minus_L(params):
    mu, sigma = params
    return -product_of_integrals(degres, intervals, mu, sigma)
intervals = [
    [-np.inf, 1], [1, 2], [2, 3], [3, 4], [4, 5],
    [5, 6], [6, 7], [7, 8], [8, 9], [9, np.inf]
]
degres =  np.array([5, 8, 6, 12, 14, 18, 11, 6, 13, 7])

# Начальное приближение
mean_sample = np.mean(sample)
std = np.std(sample, ddof=1)
initial_guess = [mean_sample, std]

# Поиск минимума отрицательной функции (эквивалентно поиску максимума)
result = minimize(minus_L, initial_guess, method='BFGS')

# Извлекаем результат
max_point = result.x
max_value = -result.fun  # Возвращаем исходное значение

print(f"Максимум находится в точке (mean: {max_point[0]:.3f}, std: {max_point[1]:.3f})")

Максимум находится в точке (mean: 4.770, std: 2.518)


In [71]:
import pandas as pd

def normal_distribution_integral(pair, mu, sigma):
    a, b = pair[0], pair[1]
    if sigma <= 0:
        raise ValueError("sigma должно быть > 0")
    return norm.cdf(b, loc=mu, scale=sigma) - norm.cdf(a, loc=mu, scale=sigma)

new_interval = []

npi = []

for interval in intervals:
    new_interval.append([interval[0], interval[1], normal_distribution_integral(interval, 4.770, 2.518) * 100])
    npi.append(normal_distribution_integral(interval, 4.770, 2.518) * 100)
npi = np.array(npi)
df = pd.DataFrame(new_interval, columns = ['beg', 'end', 'n*p'])
print(f'delta for Chi-square: {np.sum((npi - data) ** 2 / npi)}')
print(f'p-value: {0.01825}')
df

delta for Chi-square: 16.87172399691232
p-value: 0.01825


,beg,end,n*p
0,-inf,1.0,6.716801
1,1.0,2.0,6.848075
2,2.0,3.0,10.539817
3,3.0,4.0,13.883220
4,4.0,5.0,15.651058
5,5.0,6.0,15.100638
6,6.0,7.0,12.469369
7,7.0,8.0,8.812285
8,8.0,9.0,5.329944
9,9.0,inf,4.648793


In [108]:
#  Коломгоров
#  Бутстрап
n_iterations = 50000

bootstrap_delta  = []
mean = 4.77
sigma = 2.518

sample = np.repeat(np.arange(10), data)
N = len(sample)
ks_stat, p_value_ks =  kstest(sample, 'norm', args=(mean, sigma))

delta_wave = N ** 0.5 * ks_stat
print(f'delta_wave: {delta_wave}')

for i in range(n_iterations):
    new_sample = np.random.normal(mean, sigma, N)
    bootstrap_mean = np.mean(new_sample)
    bootstrap_std_dev = np.std(new_sample) * N /(N - 1)
    ks_stat, p_value_ks = kstest(new_sample, 'norm', args=(bootstrap_mean, bootstrap_std_dev))
    bootstrap_delta.append(N ** 0.5 * ks_stat)

bootstrap_delta = np.array(bootstrap_delta)
p_val = len(bootstrap_delta[bootstrap_delta >= delta_wave]) / n_iterations
p_val

[0 0 0 0 0 1 1 1 1 1 1 1 1 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 6
 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9]
delta_wave: 1.0021263220748111


0.0153